In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np
import pandas as pd
import os
import shutil

#import glob2

#from numba import jit

colors = [
'black','blue','red','green',
'cyan',  
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']

Autosaving every 180 seconds


In [2]:
# 
temps = np.load('/media/cat/2TB/liam/512channels/2009-04-13-5_120mins/tmp/output/templates/templates_0sec.npy')
print (temps.shape)


(2413, 101, 512)
